In [1]:
import pandas as pd
from scipy.stats import chi2_contingency

# Load cleaned data
df = pd.read_csv('/Users/dhruvshah/Desktop/Data Science Material/SPRING 2025/CS 133/Final-Project /GACTT_RESULTS_ANONYMIZED_v2.csv')

# 1) Chi-square: fav_sample vs. gender
ct_gender = pd.crosstab(df['gender'], df['fav_sample'])
chi2, p, dof, _ = chi2_contingency(ct_gender)
print(f"Gender × Favorite: chi2={chi2:.1f}, p={p:.3f}")

# 2) Chi-square: fav_sample vs. age_group
ct_age = pd.crosstab(df['age_group'], df['fav_sample'])
chi2, p, dof, _ = chi2_contingency(ct_age)
print(f"Age × Favorite:    chi2={chi2:.1f}, p={p:.3f}")

Gender × Favorite: chi2=101.4, p=0.000
Age × Favorite:    chi2=70.0, p=0.000


 Self-Reported vs. Actual Preference Alignment


In [2]:
# Map: A=Light, B=Medium, C=Dark, D=Other
roast_map = {'Coffee A': 'Light', 'Coffee B': 'Medium',
             'Coffee C': 'Dark',  'Coffee D': 'Other'}

df['fav_roast'] = df['fav_sample'].map(roast_map)

# Agreement flag
df['roast_agree'] = df['fav_roast'] == df['self_pref_roast']

# Overall agreement rate
overall_rate = df['roast_agree'].mean()
print(f"Overall agreement: {overall_rate:.1%}")

# Agreement by age group
print(df.groupby('age_group')['roast_agree'].mean().sort_index().apply(lambda x: f"{x:.1%}"))

# Agreement by gender
print(df.groupby('gender')['roast_agree'].mean().sort_index().apply(lambda x: f"{x:.1%}"))

Overall agreement: 25.9%
age_group
18-24 years old    27.1%
25-34 years old    25.5%
35-44 years old    25.2%
45-54 years old    24.6%
55-64 years old    29.7%
<18 years old      53.3%
>65 years old      27.8%
Name: roast_agree, dtype: object
gender
Female                    29.1%
Male                      24.7%
Non-binary                26.2%
Other (please specify)    30.0%
Prefer not to say         27.3%
Name: roast_agree, dtype: object


Dive Into Tasting Notes

In [5]:
df = pd.read_csv('/Users/dhruvshah/Desktop/Data Science Material/SPRING 2025/CS 133/Final-Project /GACTT_RESULTS_ANONYMIZED_v2.csv')
print(df['self_pref_roast'].value_counts(dropna=False))

self_pref_roast
Light      1695
Medium     1482
Dark        389
Nordic       75
Blonde       74
Italian      22
French       21
NaN           3
Name: count, dtype: int64


In [6]:
roast_normalization = {
    'Blonde':     'Light',
    'Light':      'Light',
    'Nordic':     'Light',
    'French':     'Dark',
    'Dark':       'Dark',
    'Medium':     'Medium',
    # if there are any other unusual values, map them to 'Other' or NaN
}

# apply it
df['self_pref_roast_norm'] = df['self_pref_roast'].map(roast_normalization)

# confirm
print(df['self_pref_roast_norm'].value_counts(dropna=False))

self_pref_roast_norm
Light     1844
Medium    1482
Dark       410
NaN         25
Name: count, dtype: int64


In [7]:
# map fav_sample → roast level
roast_map = {'Coffee A':'Light','Coffee B':'Medium','Coffee C':'Dark','Coffee D':'Other'}
df['fav_roast'] = df['fav_sample'].map(roast_map)

# new agreement flag
df['roast_agree_norm'] = df['fav_roast'] == df['self_pref_roast_norm']

# overall
print("Overall agreement (normalized):", df['roast_agree_norm'].mean().round(3))

# by age
print(df.groupby('age_group')['roast_agree_norm'].mean().apply(lambda x: f"{x:.1%}"))

# by gender
print(df.groupby('gender')['roast_agree_norm'].mean().apply(lambda x: f"{x:.1%}"))

Overall agreement (normalized): 0.275
age_group
18-24 years old    27.8%
25-34 years old    27.1%
35-44 years old    26.9%
45-54 years old    26.8%
55-64 years old    31.4%
<18 years old      53.3%
>65 years old      30.0%
Name: roast_agree_norm, dtype: object
gender
Female                    30.1%
Male                      26.5%
Non-binary                27.2%
Other (please specify)    30.0%
Prefer not to say         27.3%
Name: roast_agree_norm, dtype: object


## 3. Statistical Test Results

### 3.1 Chi-Square Tests
- **Favorite × Gender**: χ² = 101.4, p < 0.001 → **significant**  
- **Favorite × Age Group**: χ² = 70.0, p < 0.001 → **significant**  

> _Interpretation:_ Both gender and age group show a statistically significant association with which coffee sample people chose as their favorite.

### 3.2 Roast-Preference Alignment (Normalized)
- **Overall agreement**: 27.5%  
- **By age group**:  
  - <18 years old: 53.3%  
  - 18–24 years old: 27.8%  
  - 25–34 years old: 27.1%  
  - 35–44 years old: 26.9%  
  - 45–54 years old: 26.8%  
  - 55–64 years old: 31.4%  
  - >65 years old: 30.0%  
- **By gender**:  
  - Female: 30.1%  
  - Male: 26.5%  
  - Non-binary: 27.2%  
  - Other (please specify): 30.0%  
  - Prefer not to say: 27.3%  

> _Interpretation:_ Only about a quarter of participants correctly predicted their true blind-taste winner, indicating a large disconnect between self-reported roast preferences and actual taste-test outcomes.

2. Dive into Tasting Notes for Qualitative Insights

Let’s see why people were surprised. Assuming you’ve carried over the tasting-note column (e.g. "Why did you like that coffee? (please be specific)"), use a simple keyword–frequency approach to surface common themes among the disagree subset:

In [8]:
import pandas as pd
from collections import Counter
import re

# Load data (with tasting_notes column)
df = pd.read_csv('/Users/dhruvshah/Desktop/Data Science Material/SPRING 2025/CS 133/Final-Project /GACTT_RESULTS_ANONYMIZED_v2.csv')


# Filter those whose normalized roast doesn’t match
disagree = df[df['roast_agree_norm']==False]

# Combine all notes into one big string
all_notes = " ".join(disagree['Why did you like that coffee? (please be specific)'].dropna().tolist()).lower()

# Simple word tokenization & cleanup
words = re.findall(r'\b[a-z]{4,}\b', all_notes)  
# Count top 20 words
top20 = Counter(words).most_common(20)
print(top20)

KeyError: 'roast_agree_norm'

In [9]:
disagree = df[df['roast_agree_norm'] == False]

KeyError: 'roast_agree_norm'